# Mask R-CNN - Human pose estimation

Inspect and visualize data loading and pre-processing code.

In [1]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
import mrcnn.model as modellib
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Root directory to hdd 
DATA_DIR = os.path.abspath("/data/hdd/")

# Directory to save logs and trained model
MODEL_DIR = os.path.join(DATA_DIR, "russales", "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(DATA_DIR, "russales", "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

Using TensorFlow backend.


## Configurations

In [2]:
class TrainConfig(coco.CocoConfig):
    GPU_COUNT=1

# MS COCO Dataset
config = TrainConfig()
COCO_DIR = os.path.join(DATA_DIR, "Datasets", "MSCOCO_2017")  # TODO: enter value here
# Load dataset
assert config.NAME == "coco"
# Training dataset
# load person keypoints dataset
train_dataset_keypoints = coco.CocoDataset(task_type="person_keypoints")
train_dataset_keypoints.load_coco(COCO_DIR, "train", year="2017")
train_dataset_keypoints.prepare()

#Validation dataset
val_dataset_keypoints = coco.CocoDataset(task_type="person_keypoints")
val_dataset_keypoints.load_coco(COCO_DIR, "val", year="2017")
val_dataset_keypoints.prepare()

print("Train Keypoints Image Count: {}".format(len(train_dataset_keypoints.image_ids)))
print("Train Keypoints Class Count: {}".format(train_dataset_keypoints.num_classes))
for i, info in enumerate(train_dataset_keypoints.class_info):
    print("{:3}. {:50}".format(i, info['name']))

print("Val Keypoints Image Count: {}".format(len(val_dataset_keypoints.image_ids)))
print("Val Keypoints Class Count: {}".format(val_dataset_keypoints.num_classes))
for i, info in enumerate(val_dataset_keypoints.class_info):
    print("{:3}. {:50}".format(i, info['name']))

loading annotations into memory...
Done (t=8.01s)
creating index...
index created!
Skeleton: (19, 2)
Keypoint names: (17,)
loading annotations into memory...
Done (t=0.26s)
creating index...
index created!
Skeleton: (19, 2)
Keypoint names: (17,)
Train Keypoints Image Count: 64115
Train Keypoints Class Count: 2
  0. BG                                                
  1. person                                            
Val Keypoints Image Count: 2693
Val Keypoints Class Count: 2
  0. BG                                                
  1. person                                            


## Create Model

In [3]:
# Create model object in training mode.
model = modellib.MaskRCNN(mode="training", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True,exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
print("Loading weights from ", COCO_MODEL_PATH)

Loading weights from  /data/hdd/russales/mask_rcnn_coco.h5


In [4]:
# Training - Stage 1
print("Train heads")
model.train(train_dataset_keypoints, val_dataset_keypoints,
            learning_rate=config.LEARNING_RATE,
            epochs=2,
            layers='heads')

print("done!")

Train heads

Starting at epoch 0. LR=0.002

Checkpoint Path: /data/hdd/russales/logs/coco20181201T1651/mask_rcnn_coco_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_keypoint_mask_conv1   (TimeDistributed)
mrcnn_keypoint_mask_bn1   (TimeDistributed)
mrcnn_keypoint_mask_conv2   (TimeDistributed)
mrcnn_keypoint_mask_bn2   (TimeDistributed)
mrcnn_keypoint_mask_conv3   (TimeDistributed)
mrcnn_keypoint_mask_bn3   (TimeDistributed)
mrcnn_keypoint_mask_conv4   (TimeDistributed)
mrcnn_keypoint_mask_bn4   (TimeDistributed)
mrcnn_keypoint_mask_conv5   (TimeDistributed)
mrcnn_keypoint_mask_bn5   (TimeDistributed)
mrcnn_

/home/russales/workspace/MaskRCNN/venv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/russales/workspace/MaskRCNN/venv/lib/python3.5/site-packages/keras/engine/training.py:2033: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/2
1000/1000 [==============================] - 921s 921ms/step - loss: 8.3424 - rpn_class_loss: 0.0306 - rpn_bbox_loss: 0.4068 - mrcnn_class_loss: 0.2311 - mrcnn_bbox_loss: 0.4108 - keypoint_mrcnn_mask_loss: 6.8874 - mrcnn_mask_loss: 0.3757 - val_loss: 7.9140 - val_rpn_class_loss: 0.0067 - val_rpn_bbox_loss: 0.3196 - val_mrcnn_class_loss: 0.1561 - val_mrcnn_bbox_loss: 0.3561 - val_keypoint_mrcnn_mask_loss: 6.6990 - val_mrcnn_mask_loss: 0.3764
Epoch 2/2
1000/1000 [==============================] - 853s 853ms/step - loss: 7.5221 - rpn_class_loss: 0.0258 - rpn_bbox_loss: 0.3904 - mrcnn_class_loss: 0.2120 - mrcnn_bbox_loss: 0.3356 - keypoint_mrcnn_mask_loss: 6.2224 - mrcnn_mask_loss: 0.3358 - val_loss: 7.0936 - val_rpn_class_loss: 0.0074 - val_rpn_bbox_loss: 0.2295 - val_mrcnn_class_loss: 0.1476 - val_mrcnn_bbox_loss: 0.3537 - val_keypoint_mrcnn_mask_loss: 6.0625 - val_mrcnn_mask_loss: 0.2930
done!


In [5]:
print("Training Resnet layer 4+")
model.train(train_dataset_keypoints, val_dataset_keypoints,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=8,
            layers='4+')

Training Resnet layer 4+

Starting at epoch 2. LR=0.0002

Checkpoint Path: /data/hdd/russales/logs/coco20181201T1651/mask_rcnn_coco_{epoch:04d}.h5
Selecting layers to train
res4a_branch2a         (Conv2D)
bn4a_branch2a          (BatchNorm)
res4a_branch2b         (Conv2D)
bn4a_branch2b          (BatchNorm)
res4a_branch2c         (Conv2D)
res4a_branch1          (Conv2D)
bn4a_branch2c          (BatchNorm)
bn4a_branch1           (BatchNorm)
res4b_branch2a         (Conv2D)
bn4b_branch2a          (BatchNorm)
res4b_branch2b         (Conv2D)
bn4b_branch2b          (BatchNorm)
res4b_branch2c         (Conv2D)
bn4b_branch2c          (BatchNorm)
res4c_branch2a         (Conv2D)
bn4c_branch2a          (BatchNorm)
res4c_branch2b         (Conv2D)
bn4c_branch2b          (BatchNorm)
res4c_branch2c         (Conv2D)
bn4c_branch2c          (BatchNorm)
res4d_branch2a         (Conv2D)
bn4d_branch2a          (BatchNorm)
res4d_branch2b         (Conv2D)
bn4d_branch2b          (BatchNorm)
res4d_branch2c         

/home/russales/workspace/MaskRCNN/venv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/russales/workspace/MaskRCNN/venv/lib/python3.5/site-packages/keras/engine/training.py:2033: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 3/8
1000/1000 [==============================] - 971s 971ms/step - loss: 7.2145 - rpn_class_loss: 0.0236 - rpn_bbox_loss: 0.3859 - mrcnn_class_loss: 0.1987 - mrcnn_bbox_loss: 0.3030 - keypoint_mrcnn_mask_loss: 5.9822 - mrcnn_mask_loss: 0.3211 - val_loss: 7.9834 - val_rpn_class_loss: 0.0186 - val_rpn_bbox_loss: 0.5841 - val_mrcnn_class_loss: 0.2411 - val_mrcnn_bbox_loss: 0.4553 - val_keypoint_mrcnn_mask_loss: 6.3033 - val_mrcnn_mask_loss: 0.3811
Epoch 4/8
1000/1000 [==============================] - 941s 941ms/step - loss: 7.0967 - rpn_class_loss: 0.0222 - rpn_bbox_loss: 0.3586 - mrcnn_class_loss: 0.1887 - mrcnn_bbox_loss: 0.2901 - keypoint_mrcnn_mask_loss: 5.9211 - mrcnn_mask_loss: 0.3160 - val_loss: 7.7850 - val_rpn_class_loss: 0.0156 - val_rpn_bbox_loss: 0.5200 - val_mrcnn_class_loss: 0.2180 - val_mrcnn_bbox_loss: 0.4676 - val_keypoint_mrcnn_mask_loss: 6.1773 - val_mrcnn_mask_loss: 0.3865
Epoch 5/8
1000/1000 [==============================] - 940s 940ms/step - loss: 7.0055 - rp

ERROR:root:Error processing image {'path': '/data/hdd/Datasets/MSCOCO_2017/train2017/000000202582.jpg', 'id': 202582, 'source': 'coco', 'annotations': [{'image_id': 202582, 'keypoints': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'id': 199488, 'num_keypoints': 0, 'bbox': [589.65, 204.19, 27.03, 33.31], 'area': 280.60025, 'category_id': 1, 'segmentation': [[597.61, 234.84, 598.09, 225.91, 605.33, 225.67, 611.61, 225.91, 615.71, 220.6, 616.68, 212.64, 613.78, 207.33, 610.64, 204.19, 607.99, 205.88, 605.82, 210.95, 609.68, 212.16, 607.75, 216.74, 605.58, 221.33, 595.92, 217.47, 593.27, 231.47, 590.37, 236.05, 589.65, 237.5, 593.99, 236.05, 597.61, 233.64]], 'iscrowd': 0}, {'image_id': 202582, 'keypoints': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'id': 263255, 'num_key

1000/1000 [==============================] - 938s 938ms/step - loss: 6.8649 - rpn_class_loss: 0.0196 - rpn_bbox_loss: 0.3352 - mrcnn_class_loss: 0.1720 - mrcnn_bbox_loss: 0.2647 - keypoint_mrcnn_mask_loss: 5.7738 - mrcnn_mask_loss: 0.2996 - val_loss: 7.3764 - val_rpn_class_loss: 0.0138 - val_rpn_bbox_loss: 0.4949 - val_mrcnn_class_loss: 0.2258 - val_mrcnn_bbox_loss: 0.3748 - val_keypoint_mrcnn_mask_loss: 5.8772 - val_mrcnn_mask_loss: 0.3900


In [ ]:
print("Training All Layers")
model.train(train_dataset_keypoints, val_dataset_keypoints,
            learning_rate=config.LEARNING_RATE / 100,
            epochs=20,
            layers='all')

# Train human pose MSCOCO